In [2]:
#import der benötigten Python Libraries 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

import re
import emoji

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

#um die häufigsten Wörter im Wahlkampf anzeigen lassen zu können > wird zur tokenisierung etc benötigt
import nltk 
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy

In [4]:
#DataFrames der FDP

df_fdp_raw = pd.read_excel("../data/Parteien/@fdp_user_tweets_bearbeitet.xlsx")
df_fdp_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
tweets_df_clean = df_fdp_raw[['Name', 'original_tweets']]

In [5]:
import string 

def clean_text(text):
#konvertiert text zu lowercase  
 text = text.lower()
#löscht text in eckigen Klammern
 text = re.sub(r'\[.*?\]', '', text)
#löscht Punktuation + Links + @Zeichen 
 text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#löscht Wörter die Nummern enthalten
 text = re.sub(r'\w*\d\w*', '', text)
#löscht Anführungsstriche
 text = re.sub(r'„', '', text)
 text = re.sub(r'“', '', text)
 return text

def give_emoji_free_text(text): #Emojis werden aus den tweets entfernt 
    return emoji.get_emoji_regexp().sub(r'',text)

tweets_df_clean['emoji_free_tweets'] = tweets_df_clean['original_tweets'].apply(give_emoji_free_text)

#Create a new column with emji free text 
tweets_df_clean['clean_tweets'] = tweets_df_clean['emoji_free_tweets'].apply(clean_text)                               
                               
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets
0,FDP,Statement von @c_lindner zum Wahlausgang verpasst❓ Hier kannst Du es gleich noch einmal anschauen. 💪🚀\n\n#VielZuDanken #BTW21 #VielZuTun\n\nhttps://t.co/fz8Uut3GmM,Statement von @c_lindner zum Wahlausgang verpasst Hier kannst Du es gleich noch einmal anschauen. \n\n#VielZuDanken #BTW21 #VielZuTun\n\nhttps://t.co/fz8Uut3GmM,statement von clindner zum wahlausgang verpasst hier kannst du es gleich noch einmal anschauen \n\nvielzudanken vielzutun\n\n
1,FDP,Gleich das Statement von @c_lindner auf https://t.co/QwaXhSNZHR #vielzutun #btw21 #vielzudanken,Gleich das Statement von @c_lindner auf https://t.co/QwaXhSNZHR #vielzutun #btw21 #vielzudanken,gleich das statement von clindner auf httpstcoqwaxhsnzhr vielzutun vielzudanken
2,FDP,"Wir danken allen Wählerinnen und Wählern für Euer Vertrauen 👍, allen Unterstützern, allen Wahlkämpfern und Sympathisanten für Euer unermüdliches Engagement. 💪🚀 \n\n#vielzudanken #danke #vielzutun #btw21 https://t.co/t3jBPHRPcl","Wir danken allen Wählerinnen und Wählern für Euer Vertrauen , allen Unterstützern, allen Wahlkämpfern und Sympathisanten für Euer unermüdliches Engagement. \n\n#vielzudanken #danke #vielzutun #btw21 https://t.co/t3jBPHRPcl",wir danken allen wählerinnen und wählern für euer vertrauen allen unterstützern allen wahlkämpfern und sympathisanten für euer unermüdliches engagement \n\nvielzudanken danke vielzutun
3,FDP,"Werden wir das Land, das in uns steckt. Für eine starke politische Mitte wähle heute die FDP. Alle Stimmen für die Freiheit.💪 \nUnsere Lösungen: https://t.co/5J8N7ZdG7n\n\n#BTW21 #VielZuTun https://t.co/VosUmyAvqm","Werden wir das Land, das in uns steckt. Für eine starke politische Mitte wähle heute die FDP. Alle Stimmen für die Freiheit. \nUnsere Lösungen: https://t.co/5J8N7ZdG7n\n\n#BTW21 #VielZuTun https://t.co/VosUmyAvqm",werden wir das land das in uns steckt für eine starke politische mitte wähle heute die fdp alle stimmen für die freiheit \nunsere lösungen \n\n vielzutun httpstcovosumyavqm
4,FDP,"Klimapolitik muss wirksam und effizient sein. @koehler_fdp erklärt, wie es funktioniert. 👇\nhttps://t.co/yJ10pcAoLd","Klimapolitik muss wirksam und effizient sein. @koehler_fdp erklärt, wie es funktioniert. \nhttps://t.co/yJ10pcAoLd",klimapolitik muss wirksam und effizient sein koehlerfdp erklärt wie es funktioniert \n
...,...,...,...,...
284,FDP,"Irgendwann kommt der Punkt, an dem Du weißt: Wie es ist, darf es nicht bleiben. Die großen Aufgaben sind liegen geblieben. Weil sie groß sind. #vielzutun 1/4 https://t.co/LTwVvPSbYf","Irgendwann kommt der Punkt, an dem Du weißt: Wie es ist, darf es nicht bleiben. Die großen Aufgaben sind liegen geblieben. Weil sie groß sind. #vielzutun 1/4 https://t.co/LTwVvPSbYf",irgendwann kommt der punkt an dem du weißt wie es ist darf es nicht bleiben die großen aufgaben sind liegen geblieben weil sie groß sind vielzutun httpstcoltwvvpsbyf
285,FDP,Einschalten. Reinschauen! 👍 Jetzt stellen @c_lindner und @Wissing unsere Kampagne zur Bundestagswahl #BTW21 unter https://t.co/QwaXhSNZHR vor. #VielZuTun 💪🚀\n\n#Live #Livestream #SeiDabei,Einschalten. Reinschauen! Jetzt stellen @c_lindner und @Wissing unsere Kampagne zur Bundestagswahl #BTW21 unter https://t.co/QwaXhSNZHR vor. #VielZuTun \n\n#Live #Livestream #SeiDabei,einschalten reinschauen jetzt stellen clindner und wissing unsere kampagne zur bundestagswahl unter httpstcoqwaxhsnzhr vor vielzutun \n\nlive livestream seidabei
286,FDP,Gleich um 14 Uhr stellen @c_lindner und @Wissing unsere Kampagne zur Bundestagswahl #BTW21 unter https://t.co/QwaXhSNZHR vor. #VielZuTun \n\n#Live #Livestream #SeiDabei,Gleich um 14 Uhr stellen @c_lindner und @Wissing unsere Kampagne zur Bundestagswahl #BTW21 unter https://t.co/QwaXhSNZHR vor. #VielZuTun \n\n#Live #Livestream #SeiDabei,gleich um uhr stellen clindner und wissing unsere kampagne zur bundestagswahl unter httpstcoqwaxhsnzhr vor vielzutun \n\nlive livestream seidabei
287,FDP,@fraemke @c_lindner @Wiss

In [114]:
data = list(tweets_df_clean.loc[:, "clean_tweets"].values) ## data ist ein doc

In [115]:
# Die Wörter werden lemmatisiert -> ihr Ursprungsform wird betracht (ungebeugt)
# durch den Parameter "allowed_postags" filtern wir den Text und lassen nur Substatntive und Adjektive ausgeben
def lemmatization(texts, allowed_postags=["NOUN","ADJ"]):
    nlp = spacy.load("de_core_news_lg", disable=["parser", "ner"]) 
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:120])

statement wahlausgang


In [116]:
tweets_df_clean['clean_tweets_noun'] = lemmatized_texts

In [117]:
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets,clean_tweets_noun
0,FDP,Statement von @c_lindner zum Wahlausgang verpasst❓ Hier kannst Du es gleich noch einmal anschauen. 💪🚀\n\n#VielZuDanken #BTW21 #VielZuTun\n\nhttps://t.co/fz8Uut3GmM,Statement von @c_lindner zum Wahlausgang verpasst Hier kannst Du es gleich noch einmal anschauen. \n\n#VielZuDanken #BTW21 #VielZuTun\n\nhttps://t.co/fz8Uut3GmM,statement von clindner zum wahlausgang verpasst hier kannst du es gleich noch einmal anschauen \n\nvielzudanken vielzutun\n\n,statement wahlausgang
1,FDP,Gleich das Statement von @c_lindner auf https://t.co/QwaXhSNZHR #vielzutun #btw21 #vielzudanken,Gleich das Statement von @c_lindner auf https://t.co/QwaXhSNZHR #vielzutun #btw21 #vielzudanken,gleich das statement von clindner auf httpstcoqwaxhsnzhr vielzutun vielzudanken,statement
2,FDP,"Wir danken allen Wählerinnen und Wählern für Euer Vertrauen 👍, allen Unterstützern, allen Wahlkämpfern und Sympathisanten für Euer unermüdliches Engagement. 💪🚀 \n\n#vielzudanken #danke #vielzutun #btw21 https://t.co/t3jBPHRPcl","Wir danken allen Wählerinnen und Wählern für Euer Vertrauen , allen Unterstützern, allen Wahlkämpfern und Sympathisanten für Euer unermüdliches Engagement. \n\n#vielzudanken #danke #vielzutun #btw21 https://t.co/t3jBPHRPcl",wir danken allen wählerinnen und wählern für euer vertrauen allen unterstützern allen wahlkämpfern und sympathisanten für euer unermüdliches engagement \n\nvielzudanken danke vielzutun,wählerinnen wählern vertrauen unterstützern wahlkämpfern sympathisanten unermüdlich engagement
3,FDP,"Werden wir das Land, das in uns steckt. Für eine starke politische Mitte wähle heute die FDP. Alle Stimmen für die Freiheit.💪 \nUnsere Lösungen: https://t.co/5J8N7ZdG7n\n\n#BTW21 #VielZuTun https://t.co/VosUmyAvqm","Werden wir das Land, das in uns steckt. Für eine starke politische Mitte wähle heute die FDP. Alle Stimmen für die Freiheit. \nUnsere Lösungen: https://t.co/5J8N7ZdG7n\n\n#BTW21 #VielZuTun https://t.co/VosUmyAvqm",werden wir das land das in uns steckt für eine starke politische mitte wähle heute die fdp alle stimmen für die freiheit \nunsere lösungen \n\n vielzutun httpstcovosumyavqm,land starke politische mitte stimmen freiheit lösungen
4,FDP,"Klimapolitik muss wirksam und effizient sein. @koehler_fdp erklärt, wie es funktioniert. 👇\nhttps://t.co/yJ10pcAoLd","Klimapolitik muss wirksam und effizient sein. @koehler_fdp erklärt, wie es funktioniert. \nhttps://t.co/yJ10pcAoLd",klimapolitik muss wirksam und effizient sein koehlerfdp erklärt wie es funktioniert \n,klimapolitik
...,...,...,...,...,...
284,FDP,"Irgendwann kommt der Punkt, an dem Du weißt: Wie es ist, darf es nicht bleiben. Die großen Aufgaben sind liegen geblieben. Weil sie groß sind. #vielzutun 1/4 https://t.co/LTwVvPSbYf","Irgendwann kommt der Punkt, an dem Du weißt: Wie es ist, darf es nicht bleiben. Die großen Aufgaben sind liegen geblieben. Weil sie groß sind. #vielzutun 1/4 https://t.co/LTwVvPSbYf",irgendwann kommt der punkt an dem du weißt wie es ist darf es nicht bleiben die großen aufgaben sind liegen geblieben weil sie groß sind vielzutun httpstcoltwvvpsbyf,punkt groß aufgaben httpstcoltwvvpsbyf
285,FDP,Einschalten. Reinschauen! 👍 Jetzt stellen @c_lindner und @Wissing unsere Kampagne zur Bundestagswahl #BTW21 unter https://t.co/QwaXhSNZHR vor. #VielZuTun 💪🚀\n\n#Live #Livestream #SeiDabei,Einschalten. Reinschauen! Jetzt stellen @c_lindner und @Wissing unsere Kampagne zur Bundestagswahl #BTW21 unter https://t.co/QwaXhSNZHR vor. #VielZuTun \n\n#Live #Livestream #SeiDabei,einschalten reinschauen jetzt stellen clindner und wissing unsere kampagne zur bundestagswahl unter httpstcoqwaxhsnzhr vor vielzutun \n\nlive livestream seidabei,kampagne bundestagswahl livestream
286,FDP,Gleich um 14 Uhr stellen @c_lindner und @Wissing unsere Kampagne zur Bundestagswahl #BTW21 unter https://t.co/QwaXhSNZHR vor. #VielZuTun \n\n#Live #Livestream #SeiDabei,Gleich um 14 Uhr stellen @c_lindner und @Wissing unsere K

In [118]:
#stopwords werden entfernt 
#sätze werden in einzelne Wörter heruntergebochen 

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
stop_words.extend(['amp','mehr erfahren','erfahren','vielzutun','clindner','httpstcouelfxzluyy'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets_df_clean.clean_tweets_noun.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [119]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(tweets_df_clean.clean_tweets_noun, 10)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [120]:
unigram

,unigram,count
0,klimaschutz,27
1,schlussrunde,26
2,freiheit,24
3,uhr,21
4,menschen,17
5,land,15
6,staat,15
7,privat,14
8,bildung,13
9,investitionen,13


In [121]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2,2),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(tweets_df_clean.clean_tweets_noun, 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

In [122]:
bigram

,bigram,count
0,freie demokraten,8
1,privat investitionen,6
2,kubicki schlagabtausch,6
3,stimmen freiheit,5
4,mittler einkommen,5
5,tvtipp uhr,5
6,digitalisierung klimaschutz,4
7,uhr gast,4
8,politische mitte,3
9,bildung digitalisierung,3


In [123]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3,3),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.clean_tweets_noun, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [124]:
trigram

,trigram,count
0,ziel strikt sinkend,3
1,strikt sinkend umfassend,3
2,sinkend umfassend emissionshandelssystem,3
3,ziel pariser abkommen,3
4,pariser abkommen erderwärmung,3
5,kleine mittler einkommen,3
6,kampagne bundestagswahl livestream,3
7,privat investitionen klimaschutz,2
8,investitionen privat bereich,2
9,abkommen erderwärmung grad,2


In [125]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,2),#bigramme werden berücksichtigt 
                        max_df=0.8,
                        min_df=2,# minimales Vorkommen eines Wortes
                        stop_words= stop_words,#stopwords werden entfernt 
                        lowercase=True #lowercase 
                        )
data_matrix = vectorizer.fit_transform(tweets_df_clean.clean_tweets_noun)
data_matrix

<289x346 sparse matrix of type '<class 'numpy.int64'>'
	with 1272 stored elements in Compressed Sparse Row format>

In [126]:
lda_model = LatentDirichletAllocation(
n_components=5, # Nummer von Topics
learning_method='batch',
random_state=42,       
n_jobs = -1  # Alle verfügbaren CPUs werden verwendet 
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [127]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')


Top 10 words for topic #0:
['vierkampf', 'schlussrunde', 'gast', 'servicetweet', 'hoch', 'wahlprogramm', 'livestream', 'tvtipp', 'illner', 'uhr']


Top 10 words for topic #1:
['nächst', 'politik', 'impfen', 'bundestag', 'sommerinterview', 'pandemie', 'menschen', 'land', 'klimaschutz', 'freiheit']


Top 10 words for topic #2:
['menschen', 'mitte', 'privat investitionen', 'rente', 'jahr', 'steuererhöhungen', 'groß', 'investitionen', 'schlussrunde', 'privat']


Top 10 words for topic #3:
['kubicki schlagabtausch', 'kubicki', 'demokraten', 'gut', 'freie demokraten', 'zukunft', 'freie', 'freiheit', 'schlagabtausch', 'bildung']


Top 10 words for topic #4:
['digitalisierung', 'superabschreibungsprogramm', 'annewill', 'neu', 'fortschritt', 'fragen', 'klimaschutz', 'jahren', 'sozial', 'wirtschaftlich']




In [69]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builti

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  23.355279
2       51.163364 -159.968018       2        1  21.150055
0      -73.144554   21.523815       3        1  20.173367
1      -66.410133 -110.606438       4        1  18.331091
3       54.417324   12.443444       5        1  16.990208, topic_info=                 Term       Freq      Total Category  logprob  loglift
696       mindestlohn  45.000000  45.000000  Default  30.0000  30.0000
1171          zukunft  45.000000  45.000000  Default  29.0000  29.0000
693         millionen  25.000000  25.000000  Default  28.0000  28.0000
481              jahr  34.000000  34.000000  Default  27.0000  27.0000
1144        wohnungen  25.000000  25.000000  Default  26.0000  26.0000
...               ...        ...        ...      ...      ...      ...
1037              uhr   5.326528  21.550324   Topic5  -5.2499   0.3748
393               gut   6.860582  71.990388   Topic5  -4.9968  -0.5782
379              groß   5.128355  39.981996   Topic5  -5.2879  -0.2811
521   kanzlerkandidat   4.771472  25.928602   Topic5  -5.3600   0.0798
1161             zeit   4.568538  14.899165   Topic5  -5.4034   0.5904

[316 rows x 6 columns], token_table=      Topic      Freq                            Term
term                                                 
2         4  0.903768                     absicherung
17        4  0.903768                       angehörig
19        5  0.812246                        anhebung
21        5  0.912556  anhebung renteneintrittsalters
36        1  0.232590                          arbeit
...     ...       ...                             ...
1183      4  0.723127       zusammenhalt gesellschaft
1186      1  0.742570                  zuschauerinnen
1186      2  0.148514                  zuschauerinnen
1195      4  0.173775                      öffentlich
1195      5  0.695102                      öffentlich

[462 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 1, 2, 4])